# Simulate populations of neurons using NEST

In [1]:
!pip install seaborn
import seaborn as sns
from scipy.special import erf
import pylab
import numpy as np
import matplotlib.pyplot as plt
import nest
import csv



### Create populations

Each population is composed of 4,175 IAF neurons from which
80% are excitatory.

### Config

In [2]:
np.random.seed(42)

exc_cfg = {"I_e": 200.0, "tau_m": 20.0}
nest.CopyModel("iaf_psc_alpha", "exc_iaf_psc_alpha")
nest.SetDefaults("exc_iaf_psc_alpha", exc_cfg)

inh_cfg = {"I_e": 200.}

inner_connectivity = .1
outer_connectivity = .0025
pop_size = 4175

# simulation times
t_sim = 1500.
t_wu = 100.

# spike detectors
n_sds = 100

# poisson generators per neuron
# NOTE: much more than 20 are not possible with 16 GB RAM
n_pg = 5

# histograms of coincidences
n_chists = 300

# firing histogram
n_fhist = 300

In [3]:
p1 = nest.Create("exc_iaf_psc_alpha", pop_size)
p2 = nest.Create("exc_iaf_psc_alpha", pop_size)
p3 = nest.Create("exc_iaf_psc_alpha", pop_size)

In [4]:
# make 20% of the neurons inhibitory
inh_boundary = int(pop_size * .2)
for p in [p1,p2,p3]:
    for neuron in p[:inh_boundary]:
        nest.SetStatus(neuron, inh_cfg)

In [5]:
# set random start Vs for all neurons
"""
Vth=20.
Vrest=10.
V_range = Vrest + (Vth - Vrest)
for n1,n2,n3 in zip(p1,p2,p3):
    nest.SetStatus(n1, {"V_m": V_range * np.random.rand()})    
    nest.SetStatus(n2, {"V_m": V_range * np.random.rand()})
    nest.SetStatus(n3, {"V_m": V_range * np.random.rand()})    
"""

V_range = np.linspace(-77,-55,100)

nest.SetStatus(p1, {"V_m": np.random.choice(V_range)})    
nest.SetStatus(p2, {"V_m": np.random.choice(V_range)})
nest.SetStatus(p3, {"V_m": np.random.choice(V_range)})    

## First case: Connect neurons only within populations

### Connect neurons

Each neuron receives a synapse from 10% of randomly selected cells inside its population.




In [6]:
opt = np.arange(0, pop_size)
num_inner_conn = int(pop_size * inner_connectivity)
for i, (n1,n2,n3) in enumerate(zip(p1,p2,p3)):
    
    # for each neuron randomize N * .1 ~ 417 neurons to connect to
    c1 = np.random.choice(opt, num_inner_conn, replace=False)
    c2 = np.random.choice(opt, num_inner_conn, replace=False)
    c3 = np.random.choice(opt, num_inner_conn, replace=False)
     
    nest.Connect(p1[c1[c1 <835]], n1, syn_spec =  {'weight' : -1} )    
    nest.Connect(p1[c1[c1 >= 835]], n1)   
    
    nest.Connect(p2[c2[c2 <835]], n2, syn_spec =  {'weight' : -1} )    
    nest.Connect(p2[c2[c2 >= 835]], n2)       
    
    nest.Connect(p3[c3[c3 <835]], n3, syn_spec =  {'weight' : -1} )    
    nest.Connect(p3[c3[c3 >= 835]], n3)   

In [7]:
# TODO: add external poisson noise
pg_cfg = {"rate": 5.}
for i, (n1, n2, n3) in enumerate(zip(p1, p2, p3)):
    pg1 = nest.Create("poisson_generator", n_pg, pg_cfg)
    nest.Connect(pg1, n1)
    
    pg2 = nest.Create("poisson_generator", n_pg, pg_cfg)
    nest.Connect(pg2, n2)
    
    pg3 = nest.Create("poisson_generator", n_pg, pg_cfg)
    nest.Connect(pg3, n3)

### Simulate, measure and plot

In [8]:
# create a multimeter and a spike detector for each population
"""
m1 = nest.Create("multimeter")
m2 = nest.Create("multimeter")
m3 = nest.Create("multimeter")

nest.SetStatus(m1, {"time_in_steps":True, "record_from":["V_m"]})
nest.SetStatus(m2, {"time_in_steps":True, "record_from":["V_m"]})
nest.SetStatus(m3, {"time_in_steps":True, "record_from":["V_m"]})
"""

m = [nest.Create("multimeter", params={"time_in_steps":True, "record_from":["V_m"]}) for _ in range(300)]
s = [nest.Create("spike_detector") for _ in range(300)]
opt_inh = np.arange(0, 835)
opt_exc = np.arange(835, 4175)

n1_inh = np.random.choice(opt_inh, 20, replace=False)
n2_inh = np.random.choice(opt_inh, 20, replace=False)
n3_inh = np.random.choice(opt_inh, 20, replace=False)

n1_exc = np.random.choice(opt_exc, 80, replace=False)
n2_exc = np.random.choice(opt_exc, 80, replace=False)
n3_exc = np.random.choice(opt_exc, 80, replace=False)

for i in range(20):
    # inhibitory
    nest.Connect(m[99-i],p1[n1_inh[i]])
    nest.Connect(p1[n1_inh[i]], s[99-i])
    nest.Connect(m[199-i],p2[n2_inh[i]])
    nest.Connect(p2[n2_inh[i]], s[199-i])
    nest.Connect(m[299-i],p3[n3_inh[i]])
    nest.Connect(p3[n3_inh[i]], s[299-i])
    
    
for i in range(80):
    # excitatory
    nest.Connect(m[i],p1[n1_exc[i]])
    nest.Connect(p1[n1_exc[i]], s[i])
    nest.Connect(m[i+100],p2[n2_exc[i]])
    nest.Connect(p2[n2_exc[i]], s[i+100])
    nest.Connect(m[i+200],p3[n3_exc[i]])
    nest.Connect(p3[n3_exc[i]], s[i+200])
    

In [ ]:
nest.Simulate(2000)

In [ ]:
def raster_plot():

    plt.rcParams["figure.figsize"] = (20,10)

    pylab.figure()
    for i, sd in enumerate(s):
        dSD = nest.GetStatus(sd, keys="events")[0]
        ts = dSD["times"]
        if i % 100 >= 80:
            pylab.plot(ts, [i]*len(ts), ".", color="gray", markersize=2)
        else:
            pylab.plot(ts, [i]*len(ts), ".", color="black", markersize=2)


    plt.axhspan(0,80,facecolor="lightblue",alpha=0.2)
    plt.axhspan(80,100,facecolor="lightblue",alpha=0.1)
    plt.axhspan(100,180,facecolor="lightgreen",alpha=0.2)
    plt.axhspan(180,200,facecolor="lightgreen",alpha=0.1)
    plt.axhspan(200,280,facecolor="blue",alpha=0.2)
    plt.axhspan(280,300,facecolor="blue",alpha=0.1)
    pylab.title("Raster plot")
    pylab.xlabel("time in ms")
    pylab.ylabel("neuron ID")
    pylab.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20,5)

def plot_V(pops=[]):
    
    for idx, multimeter in enumerate(pops):

        dmm = nest.GetStatus(multimeter)[0]
        Vms = dmm["events"]["V_m"]
        ts = dmm["events"]["times"]
        plt.plot(ts, Vms)
    
    pylab.title("Merged V_ms")
    pylab.legend(["alhpa", "beta", "gamma"])
    pylab.xlabel("t")
    pylab.ylabel("V_m")
    pylab.show()
        
    

In [ ]:
from scipy.spatial import KDTree

def plot_spikes(sds=[]):
    pylab.figure()
    for sd in sds:
        dSD = nest.GetStatus(sd, keys="events")[0]
        ts = dSD["times"]
        my_tree = KDTree(np.reshape(ts, (len(ts), 1)))
        for sd2 in sds:
            if not np.array_equal(sd, sd2):
                dSD2 = nest.GetStatus(sd2, keys="events")[0]
                ts2 = dSD2["times"]
                nn_dist, nn_idx = my_tree.query(np.reshape(ts2, (len(ts2), 1)))
                pylab.plot(ts2[1:], nn_dist[1:])
        pylab.plot(ts, np.full((len(ts)), 1), ".")
    
    pylab.title("Spike distances between neurons")
    pylab.xlabel("t")
    pylab.ylabel("Distance")
    pylab.show()

In [ ]:
def coinc_hists():
    
    start_idx = 100
    end_idx = 400
    threshold = .008
    
    offsets = np.arange(-80, 80, 2)
    hist12 = np.empty((300, len(offsets)))
    hist13 = np.empty((300, len(offsets)))
    hist23 = np.empty((300, len(offsets)))
    
    for sample in range(300):
        
    
        idx1 = np.random.randint(100)
        idx2 = np.random.randint(100, 200)
        idx3 = np.random.randint(200, 300)

        m1 = m[idx1]
        m2 = m[idx2]
        m3 = m[idx3]

        Vms1 = nest.GetStatus(m1)[0]["events"]["V_m"]
        Vms2 = nest.GetStatus(m2)[0]["events"]["V_m"]
        Vms3 = nest.GetStatus(m3)[0]["events"]["V_m"]

        for i, offset in enumerate(offsets):
            diff12 = Vms1[start_idx : end_idx] - Vms2[start_idx + offset: end_idx + offset]
            diff13 = Vms1[start_idx : end_idx] - Vms3[start_idx + offset: end_idx + offset]
            diff23 = Vms2[start_idx : end_idx] - Vms3[start_idx + offset: end_idx + offset]
            hist12[sample, i] = len(diff12[np.abs(diff12) < threshold])
            hist13[sample, i] = len(diff13[np.abs(diff13) < threshold])
            hist23[sample, i] = len(diff23[np.abs(diff23) < threshold])
            
    hist12_mean = np.mean(hist12, axis=0)
    hist13_mean = np.mean(hist13, axis=0)
    hist23_mean = np.mean(hist23, axis=0)
        
    pylab.figure()
    pylab.title("alpha & beta")
    pylab.bar(offsets, hist12_mean)
    pylab.show()
    pylab.figure()
    pylab.title("alpha & gamma")
    pylab.bar(offsets, hist13_mean)
    pylab.show()
    pylab.figure()
    pylab.title("beta & gamma")
    pylab.bar(offsets, hist23_mean)
    pylab.show()

In [ ]:
def firing_hists():

    ts1 = []      
    ts2 = []      
    ts3 = []    
    
    for i in range(100):
        ts1.append(nest.GetStatus(s[i], keys="events")[0]["times"])
        ts2.append(nest.GetStatus(s[i+100], keys="events")[0]["times"])
        ts3.append(nest.GetStatus(s[i+200], keys="events")[0]["times"])
        
    ts1 = np.hstack(ts1)
    ts2 = np.hstack(ts2)
    ts3 = np.hstack(ts3)
    
    bin_size = 1.
    bins = np.arange(0, 500, bin_size)
    
    hist1, _ = np.histogram(ts1, bins=bins)
    hist2, _ = np.histogram(ts2, bins=bins)
    hist3, _ = np.histogram(ts3, bins=bins)
    
    bin_means = (bins - bin_size / 2)[1:]
    
    pylab.figure()
    pylab.plot(bin_means, hist1)
    pylab.plot(bin_means, hist2)
    pylab.plot(bin_means, hist3)
    pylab.legend(["alhpa", "beta", "gamma"])
    pylab.xlabel("t")
    pylab.ylabel("Number of spikes")
    pylab.title("PTSH")
    pylab.show()
    

In [ ]:
def sns_plots():

    pal = sns.cubehelix_palette(3, rot=-.5, dark=.3)
    
    idx1 = np.random.randint(100)
    idx2 = np.random.randint(100, 200)
    idx3 = np.random.randint(200, 300)

    m1 = m[idx1]
    m2 = m[idx2]
    m3 = m[idx3]

    Vms1 = nest.GetStatus(m1)[0]["events"]["V_m"]
    Vms2 = nest.GetStatus(m2)[0]["events"]["V_m"]
    Vms3 = nest.GetStatus(m3)[0]["events"]["V_m"]

    diff12 = Vms1 - Vms2
    diff13 = Vms1 - Vms3
    diff23 = Vms2 - Vms3
    
    data = [diff12, diff13, diff23]
    sns.violinplot(data=data, palette=pal)

In [ ]:
firing_hists()

In [ ]:
# plot before interpopulational connection
raster_plot()

In [ ]:
coinc_hists()

In [ ]:
sns_plots()

In [ ]:
vm1 = np.reshape([nest.GetStatus(mult)[0]["events"]["V_m"] for mult in m], (300,1999))

with open("vm1.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv,delimiter=',')
    csvWriter.writerows(vm1)

with open("sd1.csv","w+") as my_csv:    
    csvWriter = csv.writer(my_csv,delimiter=',')
    for spiked in s:
        csvWriter.writerow(nest.GetStatus(spiked)[0]["events"]["times"])
    